# Dataframes in Spark

A dataframe is a different type of datastructure that Spark contains, vs the RDD. 
Is it distributed too? 
From KDNuggets: (http://www.kdnuggets.com/2016/02/apache-spark-rdd-dataframe-dataset.html)
*Spark 1.0 used the RDD API but in the past twelve months, two new alternative and incompatible APIs have been introduced. Spark 1.3 introduced the radically different DataFrame API and the recently released Spark 1.6 release introduces a preview of the new Dataset API. 
Many existing Spark developers will be wondering whether to jump from RDDs directly to the Dataset API, or whether to first move to the DataFrame API. Newcomers to Spark will have to choose which API to start learning with. 
*

Dataframes vs RDD - DF allows you to work with the whole row, and with whole columns. 
More like pandas. See KDNuggets article. 

### R quirks: 

<- is the same as =
R's key datastructure is the dataframe which loads tabular csv like data. 

In [3]:
import pyspark as ps
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("PythonSQL")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

In [8]:
#Create a dataframe called flights: 

flights = spark.read.csv('492610425_T_ONTIME.csv',header='True')

In [19]:
print flights.schema

StructType(List(StructField(YEAR,StringType,true),StructField(UNIQUE_CARRIER,StringType,true),StructField(ORIGIN_AIRPORT_ID,StringType,true),StructField(DEST_AIRPORT_ID,StringType,true),StructField(DEP_DELAY,StringType,true),StructField(ARR_DELAY,StringType,true),StructField(CANCELLED,StringType,true),StructField(_c7,StringType,true)))


## Selecting and casting columns
Create a subset of your data as soon as you can to reduce the size. 
You want to work with the most concise form as soon as possible. 

In [48]:
#Remove that last column - actually just select the columns you want - uses an sql like syntax in the select!
# use col() to select the column and apply a cast to float. 
from pyspark.sql.functions import col

subset = flights.select(col('DEP_DELAY').cast("float"), col('ARR_DELAY').cast("float"), 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID')

In [49]:
#Cache the result for later
subset.cache

<bound method DataFrame.cache of DataFrame[DEP_DELAY: float, ARR_DELAY: float, ORIGIN_AIRPORT_ID: string, DEST_AIRPORT_ID: string]>

In [50]:
subset.take(3)

[Row(DEP_DELAY=-5.0, ARR_DELAY=7.0, ORIGIN_AIRPORT_ID=u'12478', DEST_AIRPORT_ID=u'12892'),
 Row(DEP_DELAY=-10.0, ARR_DELAY=-19.0, ORIGIN_AIRPORT_ID=u'12478', DEST_AIRPORT_ID=u'12892'),
 Row(DEP_DELAY=-7.0, ARR_DELAY=-39.0, ORIGIN_AIRPORT_ID=u'12478', DEST_AIRPORT_ID=u'12892')]


Now our data is all string in string format. Use dtype to determine the type here. 
We need to convert the times to floats. 


In [51]:
subset.dtypes

[('DEP_DELAY', 'float'),
 ('ARR_DELAY', 'float'),
 ('ORIGIN_AIRPORT_ID', 'string'),
 ('DEST_AIRPORT_ID', 'string')]

In [75]:
# Groupby by the destimation airport
groupedDestMean = subset.groupBy("DEST_AIRPORT_ID").mean("ARR_DELAY")
groupedOriginMean = subset.groupBy("ORIGIN_AIRPORT_ID").mean("DEP_DELAY")

# TBD - don't know how to display this result! mean() does not return an RDD... documentation needs to be accessed. 
#https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.GroupedData